<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math

Загрузим данные и проведем предобраотку данных как на семинаре.

In [4]:
# !kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
# !unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

In [234]:
articles_df = pd.read_csv("/content/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [235]:
interactions_df = pd.read_csv("/content/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [236]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [237]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [238]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [239]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId, 
                                                                  users_with_enough_interactions_df)]

In [240]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [241]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


In [242]:
# Удобно закодируем user и item
# interactions_full_df['personId'] = interactions_full_df['personId'].astype('category').cat.codes
# interactions_full_df['contentId'] = interactions_full_df['contentId'].astype('category').cat.codes

# interactions_full_df.head()

Разобьём выборку на обучение и контроль по времени.

In [243]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [244]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions['personId'] = interactions.index

interactions.head(3)

,true_train,true_test,personId
personId,,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72...",-1007001694607905623
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -...",-1032019229384696495
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -...",-108842214936804958


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [17]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 29.7 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705385 sha256=5098e165e7bef6d842f4d204134c33357129024cfd5d33353c42add8725e4712
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [18]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [322]:
from lightfm.data import Dataset

In [323]:
users = interactions_full_df['personId'].values
items = interactions_full_df['contentId'].values

In [324]:
def make_sparse(users, items):
    mat_train = Dataset()
    mat_test = Dataset()

    mat_train.fit(users, items)
    mat_test.fit(users, items)

    dct_train = [(x[0], x[1], x[2]) for x in interactions_train_df[['personId', 'contentId', 'eventStrength']].values]
    dct_test = [(x[0], x[1], x[2]) for x in interactions_test_df[['personId', 'contentId', 'eventStrength']].values]
    
    _, data_train = mat_train.build_interactions(dct_train)
    _, data_test = mat_test.build_interactions(dct_test)
    return data_train, data_test

In [325]:
data_train, data_test = make_sparse(users, items)
data_train, data_test

(<1140x2984 sparse matrix of type '<class 'numpy.float32'>'
 	with 29329 stored elements in COOrdinate format>,
 <1140x2984 sparse matrix of type '<class 'numpy.float32'>'
 	with 9777 stored elements in COOrdinate format>)

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [367]:
np.random.seed(1)

In [368]:
lfm = LightFM(loss="warp")

lfm.fit(data_train, epochs=20)
precision_at_k(lfm, data_test, k=10).mean()

0.00407332

## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [251]:
articles_df.columns

Index(['timestamp', 'eventType', 'contentId', 'authorPersonId',
       'authorSessionId', 'authorUserAgent', 'authorRegion', 'authorCountry',
       'contentType', 'url', 'title', 'text', 'lang'],
      dtype='object')

In [419]:
art = articles_df[['contentId', 'title', 'text', 'lang']].copy()
art['text'] = art['lang'] + art['title'] + art['text']
art = art.drop('title', axis=1)

In [420]:
art.head()

,contentId,text,lang
1,-4110354420726924665,"enEthereum, a Virtual Currency, Enables Transa...",en
2,-7292285110016212249,enBitcoin Future: When GBPcoin of Branson Wins...,en
3,-6151852268067518688,enGoogle Data Center 360° TourWe're excited to...,en
4,2448026894306402386,"enIBM Wants to ""Evolve the Internet"" With Bloc...",en
5,-2826566343807132236,enIEEE to Talk Blockchain at Cloud Computing O...,en


In [316]:
items_to_save = []
for a in art.values:
    if a[0] in np.unique(items):
        items_to_save.append(a[0])

len(items_to_save)

2976

In [437]:
art = art[art['contentId'].isin(items_to_save)]
art.shape

(2976, 3)

In [351]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
feat = tfidf.fit_transform(art['text'])

In [352]:
import scipy.sparse as sp

feat = sp.vstack((feat, sp.coo_matrix((8, 74472), dtype='float64')))
feat

<2984x74472 sparse matrix of type '<class 'numpy.float64'>'
	with 1047778 stored elements in COOrdinate format>

In [449]:
lfm = LightFM(loss="warp")

lfm.fit(data_train, item_features=feat, epochs=5)
precision_at_k(lfm, data_test, item_features=feat, k=10).mean()

0.004378819

## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [393]:
import re
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from string import punctuation


In [429]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [384]:
art['lang'].unique()

array(['en', 'pt', 'es', 'la', 'ja'], dtype=object)

In [404]:
art[art['lang'] == 'ja']

,contentId,lang,text
1825,-5527182266336855540,ja,40 Basic Japanese conversationsJapanese conver...
1851,-4434534460030275781,ja,"The Algorithm March, Japan's Strangely Enterta..."


In [421]:
len(stopwords.words(['english', 'portuguese']))

383

In [426]:
def tokenizer(text: str) -> list:
    """
    Функция оставляет слова и токенизирует их.
    """
    lang = text[:2]
    # Preparaition
    tok_text = word_tokenize(text.lower())
    
    # Filtration
    noise = stopwords.words(['english', 'portuguese']) + list(punctuation)
    filt_text = [w for w in tok_text if w[0] not in noise and not w[0].isdigit()]
    
    # Stemming
    if lang == 'pt':
        snowball = SnowballStemmer('portuguese')
    else:
        snowball = SnowballStemmer('english')
    
    res_text = [snowball.stem(w) for w in filt_text]
    return res_text

In [441]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf2 = TfidfVectorizer(tokenizer=tokenizer)
feat2 = tfidf2.fit_transform(art['text'])

In [443]:
feat2

<2976x32048 sparse matrix of type '<class 'numpy.float64'>'
	with 452269 stored elements in Compressed Sparse Row format>

In [444]:
feat3 = sp.vstack((feat2, sp.coo_matrix((8, 32048), dtype='float64')))
feat3

<2984x32048 sparse matrix of type '<class 'numpy.float64'>'
	with 452269 stored elements in COOrdinate format>

In [448]:
lfm = LightFM(loss="warp")

lfm.fit(data_train, item_features=feat3, epochs=5)
precision_at_k(lfm, data_test, item_features=feat3, k=10).mean()

0.0038696537

Улучшилось ли качество предсказания? Качество ухудшилось, возможно из-за random.seed. 

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [452]:
for n in [5, 10, 20, 50, 100]:
    lfm = LightFM(no_components=n, loss="warp")

    lfm.fit(data_train, item_features=feat3, epochs=5)
    prec = precision_at_k(lfm, data_test, item_features=feat3, k=10).mean()
    print(n, prec)

5 0.0030549897
10 0.004276986
20 0.0030549897
50 0.0035641547
100 0.004378819


## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь